In [1]:
import requests
import json
from pprint import pprint
import os
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import librosa
import librosa.effects
import soundfile as sf
import pyrubberband



In [2]:
'''How to reqest Access Token (Expires every hour)

In Terminal: 

curl -X POST "https://accounts.spotify.com/api/token" \ 

 -H "Content-Type: application/x-www-form-urlencoded" \ 

-d "grant_type=client_credentials&client_id=938ca3beb904426fb7623368c3024a1d&client_secret=57e5d52ae0134fb8b03b6fd8ec1b056d"
'''

'How to reqest Access Token (Expires every hour)\n\nIn Terminal: \n\ncurl -X POST "https://accounts.spotify.com/api/token" \\ \n\n -H "Content-Type: application/x-www-form-urlencoded" \\ \n\n-d "grant_type=client_credentials&client_id=938ca3beb904426fb7623368c3024a1d&client_secret=57e5d52ae0134fb8b03b6fd8ec1b056d"\n'

In [3]:
# {"access_token":"BQCb6mPAxrGI0hxw43upQWdI-spnc_AMrpRSIJqfmc41MFVuIYaHSCt20cWZG88ZaYeAArnct8cM90DeQ6d_5J2ZcB9-RbVrcOQXztjufmKM-Try-kZB","token_type":"Bearer",
# "expires_in":3600} 

In [4]:

# replace with your own API credentials
client_id = '938ca3beb904426fb7623368c3024a1d'
client_secret = '57e5d52ae0134fb8b03b6fd8ec1b056d'
token_url = 'https://accounts.spotify.com/api/token'




In [5]:

auth_response = requests.post(token_url, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']


In [6]:
#Testing my access token 
artist_id = '4Z8W4fKeB5YxbusRsdQVPb'

headers = {
    'Authorization': f'Bearer {access_token}'
}

response = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}', headers=headers)

if response.status_code == 200:
    artist_data = response.json()
    pprint(artist_data)
else:
    print(f'Request failed with status code {response.status_code}')


{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
 'followers': {'href': None, 'total': 8044717},
 'genres': ['alternative rock',
            'art rock',
            'melancholia',
            'oxford indie',
            'permanent wave',
            'rock'],
 'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
 'id': '4Z8W4fKeB5YxbusRsdQVPb',
 'images': [{'height': 640,
             'url': 'https://i.scdn.co/image/ab6761610000e5eba03696716c9ee605006047fd',
             'width': 640},
            {'height': 320,
             'url': 'https://i.scdn.co/image/ab67616100005174a03696716c9ee605006047fd',
             'width': 320},
            {'height': 160,
             'url': 'https://i.scdn.co/image/ab6761610000f178a03696716c9ee605006047fd',
             'width': 160}],
 'name': 'Radiohead',
 'popularity': 81,
 'type': 'artist',
 'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}


# Grabbing and Downloading a song from spotify

In [7]:


# Set up your Spotify API credentials

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Set the search query (song title and/or artist name)
query = 'Adrenaline by Chipbagov'

# Search for the song and extract the track name, artist, and id
results = sp.search(q=query, limit=1, type='track')
if results['tracks']['items']:
    track = results['tracks']['items'][0]
    track_name = track['name']
    artist_name = track['artists'][0]['name']
    track_id = track['id']
    print(f'Found track: {track_name} by {artist_name} (id={track_id})')
else:
    print(f'Could not find track matching query "{query}"')
    exit()

# Get the tempo of the original song
audio_features = sp.audio_features(track_id)[0]
tempo = audio_features['tempo']
print(f'Original tempo: {tempo} BPM')

# Download the original song as a WAV file
url = track['preview_url']
response = requests.get(url)
audio_data = response.content
with open('original_song.wav', 'wb') as f:
    f.write(audio_data)



Found track: Adrenaline by chipbagov (id=0kLuOdcYcXQm83O9YPGlFs)
Original tempo: 114.992 BPM


# Modulating the tempo

In [10]:

# Set the tempo scaling factor (1.0 = no change, 2.0 = double speed, 0.5 = half speed)
tempo_factor = 1.2

# Load the audio file
audio, sr = sf.read('original_song.wav')

# Stretch the audio file to the desired tempo
stretched_audio = pyrubberband.time_stretch(audio, sr, tempo_factor)

# Save the modified audio file
sf.write('modified_song2.wav', stretched_audio, sr)


# Playing the song

In [11]:
#playing the song

from pydub import AudioSegment
from pydub.playback import play

# Load the modified song into pydub
song = AudioSegment.from_wav('modified_song2.wav')

# Play the song
play(song)

Input #0, wav, from '/var/folders/pj/c3bwxcz17fx4v5_bp1h430h00000gn/T/tmpaur6p5pp.wav':
  Duration: 00:00:24.76, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s


KeyboardInterrupt: 

In [ ]:
#tempo (in BPM) = (running pace (in steps per minute) * 2) / 3
